In [78]:
import json
import pandas as pd
import numpy as np

with open("SAMBA_bilayers_database.json", encoding='utf-8') as file:
    results = json.load(file)

from pymatgen.core.periodic_table import Element

def get_atomic_number(string):
    # Get the element object for a specific element
    element = Element(string)
    # Retrieve the atomic number
    atomic_number = element.Z
    #print(f"The atomic number of {element.symbol} is {atomic_number}.")
    return atomic_number

def get_van_der_waals_radius(string):
    # Get the element object for a specific element
    element = Element(string)
    # Retrieve the atomic radius
    van_der_waals_radius = element.van_der_waals_radius
    #print(f"The atomic radius of {element.symbol} is {van_der_waals_radius}.")
    return float(van_der_waals_radius)


line = 0
for result in results:
    object_ = result
    print(f"Line:{line}")
    print(f"ID:{object_['id']}")
    print(f"Formula:{object_['formula']}")
    print(f"Target Z distance:{object_['z_separation'][0]}")
    print(object_['type_ions_layers'])
    tp_ions_layer = np.concatenate(object_['type_ions_layers']).tolist()
    print(tp_ions_layer)
    Z_array = np.array([])
    for i in tp_ions_layer:
        Z = get_atomic_number(i)
        r = get_atomic_radius(i)
        print(f"Type_ions_layer: {i}(Z:{Z};r:{r} Ang)")
        Z_array = np.hstack((Z_array, Z))
    print(f"Z_array:{Z_array}")
    print(f"Z_med_total:{np.mean(Z_array)}")
    print(f"Z_max_total:{np.max(Z_array)}")
    print(f"Z_min_total:{np.min(Z_array)}")
    print(f"Z_median_total:{np.median(Z_array)}")
    print("-"*10)
    line += 1
    # Print columns of structure
    #print(object_.keys())

    # Unchange all columns from .json
    if line==0:
        df = None
        df = pd.DataFrame.from_dict(object_,orient='index').T
        #df2 = pd.DataFrame({"Z_med":[np.mean(Z_array)]})
        #df = pd.concat([df,df2],axis=1)
    else:
        data = pd.DataFrame.from_dict(object_,orient='index').T
        #df2 = pd.DataFrame({"Z_med":[np.mean(Z_array)]})
        #line = pd.concat([data,df2],axis=1)
        df = pd.concat([df,data],axis=1)
    if line>=10:
        break
        
df.to_csv('output_bylayers_.csv',sep=';', index=False)

Line:0
ID:HgTe+HgTe_5666b5bd1a1fee47+0001
Formula:HgTe+HgTe
Target Z distance:3.1625449892108755
[['Hg', 'Te'], ['Hg', 'Te']]
['Hg', 'Te', 'Hg', 'Te']
Type_ions_layer: Hg(Z:80;r:1.5 Ang)
Type_ions_layer: Te(Z:52;r:1.4 Ang)
Type_ions_layer: Hg(Z:80;r:1.5 Ang)
Type_ions_layer: Te(Z:52;r:1.4 Ang)
Z_array:[80. 52. 80. 52.]
Z_med_total:66.0
Z_max_total:80.0
Z_min_total:52.0
Z_median_total:66.0
----------
Line:1
ID:ReSe2+TaSe2_b7c464cdb704bfca+0002
Formula:ReSe2+TaSe2
Target Z distance:3.0463750729290204
[['Re', 'Se'], ['Ta', 'Se']]
['Re', 'Se', 'Ta', 'Se']
Type_ions_layer: Re(Z:75;r:1.35 Ang)
Type_ions_layer: Se(Z:34;r:1.15 Ang)
Type_ions_layer: Ta(Z:73;r:1.45 Ang)
Type_ions_layer: Se(Z:34;r:1.15 Ang)
Z_array:[75. 34. 73. 34.]
Z_med_total:54.0
Z_max_total:75.0
Z_min_total:34.0
Z_median_total:53.5
----------


KeyboardInterrupt: 

Defaulting to user installation because normal site-packages is not writeable
  Using cached pymatgen-2024.8.9-cp39-cp39-win_amd64.whl (3.6 MB)
  Using cached monty-2025.1.9-py3-none-any.whl (51 kB)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'C:\Program Files\Python39\python.exe -m pip install --upgrade pip' command.


In [ ]:
import json
import numpy as np

with open('project.json', encoding='utf-8') as f:
    labels = json.load(f)

import pandas as pd

def get_rotulo(label):
    annotations = label["annotations"]
    for annotation in label["annotations"]:
        #12 -> maxieni
        #11 -> poliane
        if annotation["completed_by"]==12:
            print("Completed_by",annotation["completed_by"])
            #print(annotations)
            #print(annotations[0].keys())
            user_result = annotations[0]["result"]
            #print(user_result)
            value = user_result[0]["value"]["choices"][0]
            #print("Categoria:",value)
            if value!="Não é ataque":
                print("Quantidade results:",len(user_result))
                if len(user_result)<2:
                    return "None","None"
                value_sub = user_result[1]["value"]["choices"][0]
                #print("Sub-categoria:",value_sub)
                if value == "Ataque Moderado":
                    if value_sub == "Descredibilização por Ideologia" or value_sub == "Descredibilização por Ataques políticos":
                        value_sub = dictionary_sub[value_sub]
            else:
                value_sub = "Nenhum"
            return value,value_sub
        else:
            return "None","None"

init = True
x = 0
for label in labels:
    #print(label["data"])
    # iterating both key and values
    #for key, value in label["data"].items():
    #    print(f"{key}: {value}")
    value, value_sub = get_rotulo(label)
    if value=="None":
        x = x + 1
        continue
    dictionary = {
                    "Não é ataque":"Não é ataque",
                    "Ataque Grave":"Ataque Grave",
                    "Ataque Moderado":"Ataque Moderado",
                    "Ataque Leve":"Ataque Leve"
                 }
    print(f"ID: {label['id']}, Rótulo:{value}, Sub-rótulo:{value_sub}")
    if init==True:
        df = pd.DataFrame.from_dict(label["data"],orient='index').T
        df2 = pd.DataFrame({"rotulo":[value]})
        df = pd.concat([df,df2],axis=1)
        df3 = pd.DataFrame({"sub_rotulo":[value_sub]})
        df = pd.concat([df,df3],axis=1)
    if init == False:
        data = pd.DataFrame.from_dict(label["data"],orient='index').T
        df2 = pd.DataFrame({"rotulo":[value]})
        data = pd.concat([data,df2],axis=1)
        df3 = pd.DataFrame({"sub_rotulo":[value_sub]})
        data = pd.concat([data,df3],axis=1)
        df = pd.concat([df,data])
        
    init = False
    x = x + 1
    print(f"Processed:{x}")

df.to_csv('output_maxieni.csv', index=False)

In [24]:
print(object_.keys())

dict_keys(['id', 'number_layers', 'id_layers', 'formula', 'type_ions_layers', 'number_ions_layers', 'number_type_ions_layers', 'range_ions_layers', 'number_ions', 'area_perc_mismatch', 'perc_area_change', 'perc_mod_vectors_change', 'angle_perc_mismatch', 'perc_angle_change', 'rotation_angle', 'supercell_matrix', 'deformation_matrix', 'strain_matrix', 'shift_plane', 'z_separation', 'thickness', 'total_thickness', 'lattice_type', 'point_group', 'space_group', 'space_group_number', 'inversion_symmetry', 'structural_optimization', 'pseudo_type', 'exchange_correlation_functional', 'vdW', 'non_collinear', 'spin_orbit', 'a1', 'a2', 'a3', 'module_a1_a2_a3', 'angle_a1a2_a1a3_a2a3', 'cell_area', 'b1', 'b2', 'b3', 'module_b1_b2_b3', 'angle_b1b2_b1b3_b2b3', 'zb_area', 'direct_coord_ions', 'kpath', 'lorbit', 'ispin', 'nk', 'nb', 'ne', 'gap', 'e_vbm', 'e_cbm', 'vbm', 'cbm', 'type_gap', 'k_vbm', 'k_cbm', 'e_fermi', 'e_vacuum', 'total_energy', 'e_per_ion', 'e_per_area', 'e_binding', 'e_slide', 'charge